In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

### 学术加速

如果是在 AutoDL 租的服务器，下载模型前运行下面 cell 以获取学术加速

In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
# 查看 GPU
!nvidia-smi

No devices were found


## Qwen2.5-VL-finetune

In [2]:
import torch

MAX_LENGTH = 8192
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

### Download model

In [3]:
model_id_3b = 'Qwen/Qwen2.5-VL-3B-Instruct-AWQ'
model_id_7b = 'Qwen/Qwen2.5-VL-7B-Instruct-AWQ'
save_dir_3b = './model/base/vl3b/'  # change to your save path
save_dir_7b = './model/base/vl7b/' 

In [4]:
from huggingface_hub import snapshot_download

# snapshot_download(repo_id='Qwen/Qwen2.5-VL-3B-Instruct-AWQ', local_dir='./vl3b/', local_dir_use_symlinks=False)
snapshot_download(repo_id=model_id_7b, local_dir=save_dir_7b, local_dir_use_symlinks=False)

/root/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

'/root/autodl-tmp/HOCR/finetune/model/base/vl7b'

In [5]:
import transformers
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, AutoTokenizer
from qwen_vl_utils import process_vision_info


model_7b = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    save_dir_7b, torch_dtype=torch.float16, 
    attn_implementation="flash_attention_2", device_map='auto',
)
model_7b.to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(save_dir_7b)
processor = AutoProcessor.from_pretrained(save_dir_7b)

model_7b.enable_input_require_grads()   # 开启梯度检查点时(training_args.gradient_checkpointing=True,)要执行该方法

[2025-04-16 12:21:42,912] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/minicond

ImportError: /root/miniconda3/bin/../lib/libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /root/miniconda3/lib/python3.12/site-packages/intel_extension_for_pytorch/lib/libintel-ext-pt-cpu.so)

In [9]:
print(model_7b)

Qwen2_5_VLForConditionalGeneration(
  (visual): Qwen2_5_VisionTransformerPretrainedModel(
    (patch_embed): Qwen2_5_VisionPatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2_5_VLVisionBlock(
        (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
        (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
        (attn): Qwen2_5_VLVisionFlashAttention2(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): Qwen2_5_VLMLP(
          (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
          (up_proj): Linear(in_features=1280, out_features=3420, bias=True)
          (down_proj): Linear(in_features=3420, out_features=1280, bias=True)
          (act_fn): SiLU()
        )
      )
    )
    (merger): Qwen2_5_VLPatchMerg

### 数据集加载、划分、映射

In [10]:
def preprocess_func(example):
    # print(f'example.ids: {example,keys()}'
    input_ids, attention_mask, labels = [], [], []
    url = example["message"][0]["conversation"][0]['url']
    caption = example["message"][0]["conversation"][1]['caption']
    
    messages = [
        {
            "role": "system", 
            "content": "You are a helpful assistant in recognize math equations in either handwritten or printed text."
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text", "text": "Recognize the equation in the image, directly write its LaTeX code between `<start_latex>` and `<end_latex>` without other words."
                },
                {
                    "type": "image",
                    "image": url,
                    "resized_height": 280,
                    "resized_width": 280,
                },
            ]
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": caption
                }
            ]
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    img_inputs, _ = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=img_inputs,
        padding=True,
        return_tensors='pt'
    )
    inputs = {key: value.tolist() for key, value in inputs.items()}
    instruction = inputs
    response = tokenizer(f'{caption}', add_special_tokens=False)
    input_ids = (
        instruction["input_ids"][0] + response['input_ids'] + [tokenizer.pad_token_id]
    )
    attention_mask = instruction['attention_mask'][0] + response['attention_mask'] + [1]
    labels = (
        [-100] * len(instruction['input_ids'][0])
        + response['input_ids']
        + [tokenizer.pad_token_id]
    )

    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    labels = torch.tensor(labels)

    inputs['pixel_values'] = torch.tensor(inputs['pixel_values'])
    # 由 (1, h, w) 变换为 (h, w)
    inputs['image_grid_thw'] = torch.tensor(inputs['image_grid_thw']).squeeze(0)  
    return {
        "input_ids": input_ids, 
        "attention_mask": attention_mask, 
        "labels": labels,
        "pixel_values": inputs['pixel_values'], 
        "image_grid_thw": inputs['image_grid_thw']
    }

In [14]:
""" 数据集准备 """
import json
import random
from datasets import load_dataset, Dataset

# change cache
import os
os.environ["HF_DATASETS_CACHE"] = "./data/cache/"

dataset_dir = './ft_data.json'
dataset = load_dataset('json', data_files=dataset_dir)  # load dataset
dataset = dataset['train'].train_test_split(test_size=0.15, shuffle=True, seed=5525)  # split
# dataset.save_to_disk('data/ft_dataset')  # save dataset

training_dataset, test_dataset = dataset['train'], dataset['test']
training_dataset = training_dataset.map(preprocess_func)  # mapping training dataset

training_dataset, test_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7508 [00:00<?, ? examples/s]

(Dataset({
     features: ['message', 'input_ids', 'attention_mask', 'labels', 'pixel_values', 'image_grid_thw'],
     num_rows: 7508
 }),
 Dataset({
     features: ['message'],
     num_rows: 1326
 }))

In [15]:
training_dataset, test_dataset

(Dataset({
     features: ['message', 'input_ids', 'attention_mask', 'labels', 'pixel_values', 'image_grid_thw'],
     num_rows: 7508
 }),
 Dataset({
     features: ['message'],
     num_rows: 1326
 }))

### 配置 swanlab

如果想记录下训练可视化数据，可以用 swanlab，没有在下面 Trainer 里把 `callbacks=[swanlab_callback],  # 没有注释即可` 注释即可，同时设置 `test_results` 函数的 `return_list=False`

In [18]:
import swanlab
from swanlab.integration.transformers import SwanLabCallback

# 设置SwanLab回调
swanlab_callback = SwanLabCallback(
    project="Qwen2.5-VL-7b-new-data",
    experiment_name="qwen2.5-vl-crohme-train",
    config={
        "model": "https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct-AWQ",
        "dataset": "https://huggingface.co/datasets/RealWonster/Handwriting_Math_Equations",
        "github": "https://github.com/Wooonster/HOCR",
        "prompt": "Recognize the equation in the image, directly write its LaTeX code between `<start_latex>` and `<end_latex>` without other words.",
        "train_data_number": len(training_dataset),
        "lora_rank": 8,
        "lora_alpha": 16,
        "lora_dropout": 0.1,
    },
)

### 预测、测试函数

In [19]:
from test_funcs import compute_bleu, compute_exprate
from collections import defaultdict

In [20]:
def predict(messages, model):
    # 准备推理
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)

    # 生成输出
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return output_text[0]

def test_results(dataset, model, return_list=False):
    # test_dataset = dataset['test']
    test_outputs = []
    swan_list = []
    
    for item in dataset:
        url = item["message"][0]["conversation"][0]['url']
        caption = item["message"][0]["conversation"][1]['caption']
        # Create the conversation prompt
        messages = [
            {
                "role": "system", 
                "content": "You are a helpful assistant in recognizing math equations in either handwritten or printed text."
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Recognize the equation in the image, directly write its LaTeX code between `<start_latex>` and `<end_latex>` without other words."
                    },
                    {
                        "type": "image",
                        "image": url,
                        "resized_height": 280,
                        "resized_width": 280,
                    },
                ]
            }
        ]
    
        # Generate a prediction using the model
        response = predict(messages, model)
        # Save the prediction keyed by image URL
        test_outputs.append([url, response, caption])
        swan_list.append(swanlab.Image(url, caption=response))
        
    # Compute evaluation metrics
    compute_bleu(test_outputs)
    compute_exprate(test_outputs)
    
    return swan_list if return_list else None

In [21]:
test_results(test_dataset, model_7b, return_list=False)

/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv3d(


--------------------------------------------------Computing BLEU--------------------------------------------------
['35===35', '\\frac{2}{\\frac{3m-2n\\times9^n-9^m}{2n-1}}===\\frac{2}{\\frac{3m-2n\\times9^{n}-9^{m}}{2n-1}}', '\\frac{1+\\frac{7}{9}t^2+\\frac{64}{945}t^4}{1+\\frac{10}{9}t^2+\\frac{5}{21}t^4}===\\frac{1+\\frac{7}{9}t^{2}+\\frac{64}{945}t^{4}}{1+\\frac{10}{9}t^{2}+\\frac{5}{21}t^{4}}', 'm=-1===m=-1', '\\left|\\frac{ax+by_0+c}{\\sqrt{a^2+b^2}}\\right|===|\\frac{ax_{0}+by_{0}+c}{\\sqrt{a^{2}+b^{2}}}|', '\\left[b^{\\frac{a}{c}}\\left(\\left(\\frac{a}{b}\\right)^2+1\\right)\\right]^{\\frac{1}{c}}===[b^{x}\\{(\\frac{a}{b})^{x}+1\\}]^{\\frac{1}{x}}', 'x^n=y===x^{n}=z', '\\sqrt{xyz}=\\sqrt{x}\\sqrt{y}\\sqrt{z}===\\sqrt{xyz}=\\sqrt{x}\\sqrt{y}\\sqrt{z}', '5\\cdot4===5.4', 'z_1^2+1^2-z_2^2+2^2===z_{1}^{2}+1^{z}-z_{2}^{2}+2^{z}', '\\mu_1=\\beta\\mu_2===u_{1}=\\betau_{2}', 'k^{3}+14k^{2}-132k+178q===k^{3}+14k^{2}-132k+1789', '5\\sqrt{5}+2\\sqrt{2}===5\\sqrt{5}+2\\sqrt{2}', '\\frac{h

### 配置 LoRA 参数

In [22]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

# 配置LoRA
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=16,
    lora_dropout=0.1,  # Dropout 比例
    bias="none",
)

# 获取LoRA模型
peft_model_7b = get_peft_model(model_7b, config)
peft_model_7b.enable_input_require_grads()

### 配置预训练参数

In [29]:
from transformers import (
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    Qwen2VLForConditionalGeneration,
    AutoProcessor,
)

# 配置训练参数
training_args = TrainingArguments(
    output_dir="./model/output/Qwen2.5-VL-7B-ft/",
    do_train=True,
    per_device_train_batch_size=4,  # total batch size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=4,
    logging_strategy='steps',
    logging_steps=10,
    logging_first_step=True,
    num_train_epochs=3,
    # eval_strategy='epoch',
    save_strategy='steps',
    # save_steps=5,
    learning_rate=1e-4,
    lr_scheduler_type='cosine',
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
    # load_best_model_at_end=True,
)

开始训练，并且稳定后，在终端 `watch -n 1 nvidia-smi` 观测一下 GPU 显存占用情况，可以记录一下

In [30]:
# 配置Trainer
trainer = Trainer(
    model=peft_model_7b,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=test_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],  # 没有注释即可
)

# 开启模型训练
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
1,0.000000


swanlab: Step 1 on key train/loss already exists, ignored.
swanlab: Step 1 on key train/grad_norm already exists, ignored.
swanlab: Step 1 on key train/learning_rate already exists, ignored.
swanlab: Step 1 on key train/epoch already exists, ignored.


/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 

In [31]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

# 配置测试参数
val_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=True,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=16,
    lora_dropout=0.1,  # Dropout 比例
    bias="none",
)

# 获取测试模型
val_peft_model = PeftModel.from_pretrained(
    model_7b, 
    model_id="./model/output/Qwen2.5-VL-7B-ft/checkpoint-5/",  # 换到自己微调好的模型 ckpt 路径
    config=val_config,
    # local_files_only=True
)

In [32]:
swan_list = test_results(test_dataset, val_peft_model, return_list=True)  # 没有 swanlab, return_list=False
if swan_list is not None:
    swanlab.log({"Prediction": swan_list})
    # 在 Jupyter Notebook 中运行时要停止SwanLab记录 需要调用swanlab.finish()
    swanlab.finish()

KeyboardInterrupt: 

In [33]:
test_results(test_dataset, val_peft_model, return_list=False) 

--------------------------------------------------Computing BLEU--------------------------------------------------
['\n35\n<<<===>>>35', '\nbgroup\\left[\\begin{array}{l}\n\\frac{2}{3m-2n\\times9^{n}-9m}\\cr\n\\hline\\cr\n2n-1\\cr\n\\end{array}\\right]\negroup\n<<<===>>>\\frac{2}{\\frac{3m-2n\\times9^{n}-9^{m}}{2n-1}}', '\nbmatrix\n1+{7over9}t^2+{64over945}t^4\n\\cr\n1+{10over9}t^2+{5over21}t^4\n\\cr\nend_bmatrix\n<<<===>>>\\frac{1+\\frac{7}{9}t^{2}+\\frac{64}{945}t^{4}}{1+\\frac{10}{9}t^{2}+\\frac{5}{21}t^{4}}', '\nm=-1\n<<<===>>>m=-1', '\nbx+cy_{0}+a\nbx+cy_{0}+a\n<<<===>>>|\\frac{ax_{0}+by_{0}+c}{\\sqrt{a^{2}+b^{2}}}|', '\nb^{a\\left\\{\\left(\\frac{a}{b}\\right)^2+1\\right\\}}<<<===>>>[b^{x}\\{(\\frac{a}{b})^{x}+1\\}]^{\\frac{1}{x}}', '\nx^a=y\n<<<===>>>x^{n}=z', '\nf_{xyz}=f_xf_yf_z\n<<<===>>>\\sqrt{xyz}=\\sqrt{x}\\sqrt{y}\\sqrt{z}', '\nb^4\n<<<===>>>5.4', '\nz_1^2+1^2-z_2^2+2^2\n<<<===>>>z_{1}^{2}+1^{z}-z_{2}^{2}+2^{z}', '\nb_1=b_2\n<<<===>>>u_{1}=\\betau_{2}', '\nk^3+14k^2-132k+